In [ ]:
# Instala as dependências
!pip install -r ../../../requirements.txt

In [ ]:
# Importações para análise e tratamento dos dados
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Forçando o pandas a mostrar todas as colunas
pd.set_option('display.max_columns', 321)
pd.set_option('display.max_rows', None)

In [ ]:
# Lendo o arquivo principal e iniciando a análise
dados_principal = pd.read_csv("../../../data/dados_cvm.csv")
dados_principal.describe()

In [ ]:
# Analisando as colunas 12 e 14 que geraram um aviso
colunas_sem_null = dados_principal[['Tipo_Prazo_Conversao_Cotas', 'Tipo_Prazo_Pagamento_Resgate']].dropna()
colunas_sem_null.head(20)

Conclusão: 12 e 14 não são úteis para análise, possuem muitos registros nulos e só dizem respeito a tipos de prazos que não afetam o risco do fundo de perda financeira

In [ ]:
# Informações sobre cada coluna
dados_principal.info(max_cols=321)

Como há muitos registros nulos, explorarei esses dados primeiro para já eliminar caso possível

In [ ]:
# Visualizando quais colunas contêm dados nulos
dados_nulos = dados_principal.isnull().sum()
dados_nulos[dados_nulos > 0]

In [ ]:
# Visualizando as colunas com dados nulos por porcentagem
porcentagem_nulos = (dados_principal.isnull().sum() / len(dados_principal)) * 100
porcentagem_nulos[porcentagem_nulos >= 45]

Quarenta e cinco porcendo de dados nulos já é um valor ruim de se trabalhar. Por isso e pela análise subjetiva de utilidade das colunas listadas acima, eliminarei todas, com exceção das duas últimas (Garantias_Valor_Total e Garantias_Percentual).

In [ ]:
# Dropando as colunas
dados_atualizados = dados_principal.drop(porcentagem_nulos[porcentagem_nulos > 49].index, axis=1)
dados_atualizados.shape

In [ ]:
# Visualizando novamente os dados nulos, excluindo as colunas dropadas, a fim de visualizar quais podem ser importantes
dados_nulos_atualizados = dados_nulos.drop(porcentagem_nulos[porcentagem_nulos > 49].index)
dados_nulos_atualizados = dados_nulos_atualizados.drop(dados_nulos_atualizados[dados_nulos_atualizados == 0].index)
dados_nulos_atualizados

Conclusões: todas, com exceção das colunas "CNPJ_Administrador", "Nome_Administrador" e "Forma_Condominio", representam valores numéricos e podem ter seus registros nulos preenchidos por zeros. Antes desse procedimento, verificarei os registros nulos de "Forma_Condominio", pois não deveriam existir.

In [ ]:
# Visualizando os fundos inconsistentes
dados_atualizados[dados_principal['Forma_Condominio'].isnull() == True]

Aparentemente, esses fundos são não padronizados ou possuem dados inconsistentes. Portanto, excluirei-os.

In [ ]:
# Retirando os fundos inconsistentes
dados_atualizados = dados_atualizados[dados_atualizados['ID_Participante'] != 87248]
dados_atualizados = dados_atualizados[dados_atualizados['ID_Participante'] != 128015]
dados_atualizados.shape

In [ ]:
# Preenchendo registros nulos com zeros
dados_atualizados.drop(['CNPJ', 'CNPJ_Administrador', 'Nome_Administrador'], inplace=True, axis=1)
dados_atualizados.fillna(0, inplace=True)
dados_atualizados.shape

Livrei-me dos dados do Administrador e do CNPJ do fundo pois não são relevantes e alterei os registros. Agora, vou verificar abaixo se eles foram alterados.

In [ ]:
# Verificando alteração dos registros nulos
dados_atualizados.isnull().sum()

Removerei também as duplicatas presentes no arquivo, os fundos exclusivos ou com cotistas vinculados por interesse único e indissociável e as colunas restantes que não possuem funcionalidade para a análise

In [ ]:
# Removendo colunas não-funcionais
indexes = ['Data_Entrega']
for i in range(199, 287):
    indexes.append(dados_atualizados.columns[i])

for i in range(163, 174):
    indexes.append(dados_atualizados.columns[i])

for i in range(130, 141):
    indexes.append(dados_atualizados.columns[i])

dados_atualizados.drop_duplicates(inplace=True)
dados_atualizados.drop(indexes, inplace=True, axis=1)
dados_atualizados = dados_atualizados.query("Fundo_Exclusivo == 'Não'")
dados_atualizados = dados_atualizados.query('Patrimonio_Liquido > 1')
dados_atualizados.shape

Com esse tratamento de dados nulos, partirei para a junção desse com os outros arquivos CSV fornecidos.

In [ ]:
# Leitura do primeiro arquivo
dados_classes = pd.read_csv("../data/dados_cvm_classes.csv")
dados_classes.describe()

In [ ]:
# checando a estrutura
dados_classes.shape

Há, nesse segundo documento, aproximadamente o triplo do número de linhas do primeiro analisado. Isso ocorre porque existem várias linhas que dizem respeito ao mesmo "SK_Documento", registrando apenas as diferentes cotas, como pode ser avaliado na célula a seguir:

In [ ]:
# checando registros únicos
dados_classes['SK_Documento'].unique().shape

Portanto, para realizar o merge desse documento com o primeiro, os dados precisam ser tratados. Começarei realizando o drop de colunas que não são úteis para a análise.

In [ ]:
# drop das colunas
dados_classes.drop(
    [
        'Valor_Total_Captado',
        'Quantidade_Cotas_Emitidas',
        'Valor_Total_Resgates',
        'Quantidade_Cotas_Resgatadas',
        'Valor_A_Pagar',
        'Quantidade_Cotas_A_Resgatar',
        'Valor_Amortizado_Cota', 
        'Valor_Total_Amortizacao'
    ],
    inplace=True,
    axis=1
)
dados_classes.info()

In [ ]:
# separando subordinadas e seniores
subordinadas = dados_classes[[classe.startswith(('Subordinada', 'Classe Subordinada')) for classe in dados_classes['Classe_Serie']]]
seniores = dados_classes[[classe.startswith('Sênior') for classe in dados_classes['Classe_Serie']]]

In [ ]:
# drop nas colunas que não têm mais funcionalidade
subordinadas.drop('Classe_Serie', inplace=True, axis=1)
seniores.drop('Classe_Serie', inplace=True, axis=1)

In [ ]:
# agrupando por "SK_Documento"
subordinadas_agrupadas = subordinadas.groupby('SK_Documento').sum()
seniores_agrupadas = seniores.groupby('SK_Documento').sum()

Como todas as colunas tiveram seus valores somados, é necessário substituí-las para encontrar seus valores individuais por cotas.

In [ ]:
# drop de colunas que estão com valores somados erroneamente
subordinadas_agrupadas.drop(
    [
        'Valor_Cota',
        'Rentabilidade',
        'Desempenho_Esperado',
        'Desempenho_Realizado'
    ],
    inplace=True,
    axis=1
)
seniores_agrupadas.drop(
    [
        'Valor_Cota',
        'Rentabilidade',
        'Desempenho_Esperado',
        'Desempenho_Realizado'
    ],
    inplace=True,
    axis=1
)

In [ ]:
# médias dos valores somados para substituição
media_valor_cota_subordinada = subordinadas.query('Quantidade_Cotas != 0').groupby('SK_Documento')['Valor_Cota'].mean().to_frame().reset_index()
media_rentabilidade_subordinada = subordinadas.query('Rentabilidade != 0').groupby('SK_Documento')['Rentabilidade'].mean().to_frame().reset_index()
media_desempenho_esperado_subordinada = subordinadas.query('Desempenho_Esperado != 0').groupby('SK_Documento')['Desempenho_Esperado'].mean().to_frame().reset_index()
media_desempenho_realizado_subordinada = subordinadas.query('Desempenho_Realizado != 0').groupby('SK_Documento')['Desempenho_Realizado'].mean().to_frame().reset_index()

media_valor_cota_seniores = seniores.query('Quantidade_Cotas != 0').groupby('SK_Documento')['Valor_Cota'].mean().to_frame().reset_index()
media_rentabilidade_seniores = seniores.query('Rentabilidade != 0').groupby('SK_Documento')['Rentabilidade'].mean().to_frame().reset_index()
media_desempenho_esperado_seniores = seniores.query('Desempenho_Esperado != 0').groupby('SK_Documento')['Desempenho_Esperado'].mean().to_frame().reset_index()
media_desempenho_realizado_seniores = seniores.query('Desempenho_Realizado != 0').groupby('SK_Documento')['Desempenho_Realizado'].mean().to_frame().reset_index()

In [ ]:
# juntando os campos de médias
medias_subordinadas = pd.merge(media_valor_cota_subordinada, media_rentabilidade_subordinada, how='inner', on='SK_Documento')
medias_subordinadas = pd.merge(medias_subordinadas, media_desempenho_esperado_subordinada, how='inner', on='SK_Documento')
medias_subordinadas = pd.merge(medias_subordinadas, media_desempenho_realizado_subordinada, how='inner', on='SK_Documento')

medias_seniores = pd.merge(media_valor_cota_seniores, media_rentabilidade_seniores, how='inner', on='SK_Documento')
medias_seniores = pd.merge(medias_seniores, media_desempenho_esperado_seniores, how='inner', on='SK_Documento')
medias_seniores = pd.merge(medias_seniores, media_desempenho_realizado_seniores, how='inner', on='SK_Documento')

In [ ]:
# merges com dados finais
subordinadas_agrupadas = pd.merge(subordinadas_agrupadas, medias_subordinadas, how='outer', on='SK_Documento')
seniores_agrupadas = pd.merge(seniores_agrupadas, medias_seniores, how='outer', on='SK_Documento')

Agora, com a tabela final tratada, renomearei os campos para o merge com o primeiro documento ("IMs.csv")

In [ ]:
# renomeando as colunas
subordinadas_agrupadas = subordinadas_agrupadas.add_suffix('_Subordinada')
seniores_agrupadas = seniores_agrupadas.add_suffix('_Senior')

subordinadas_agrupadas = subordinadas_agrupadas.rename(columns={'SK_Documento_Subordinada':'SK_Documento'})
seniores_agrupadas = seniores_agrupadas.rename(columns={'SK_Documento_Senior':'SK_Documento'})
seniores_agrupadas.head()

In [ ]:
#merge entre as duas tabelas de agrupamento
dados_classes_tratados = pd.merge(subordinadas_agrupadas, seniores_agrupadas, how='outer', on='SK_Documento')
dados_classes_tratados.shape

In [ ]:
#merge com o primeiro documento
dados_atualizados = pd.merge(dados_atualizados, dados_classes_tratados, how='inner', on='SK_Documento')
dados_atualizados.info(max_cols=188)

In [ ]:
#visualizando resultado
dados_atualizados.describe()

Abaixo, testo a hipótese de construção de uma nova fórmula de valor de risco, diferente da que a CVM nos passou.

In [ ]:
#ordenando por id e data para conseguir iterar e calcular valores não pagos a prazo
dados_atualizados["Data_Competencia"] = pd.to_datetime(dados_atualizados["Data_Competencia"], format="%Y-%m-%d")
dados_atualizados.sort_values(['ID_Participante', 'Data_Competencia'], inplace=True)
dados_atualizados.head(40)

In [ ]:
#criando colunas de valores não pagos a prazo
dados_atualizados = dados_atualizados.assign(valor_nao_pago_com_beneficios = 0)
dados_atualizados = dados_atualizados.assign(valor_nao_pago_sem_beneficios = 0)
dados_atualizados.reset_index(inplace=True)
dados_atualizados.head(20)

In [ ]:
#atribuindo valores às colunas
for i in range(1, len(dados_atualizados)):
    atual = dados_atualizados.iloc[i]
    anterior = dados_atualizados.iloc[i - 1]
    if atual.ID_Participante == anterior.ID_Participante and anterior['Carteira_Direitos_Aquisicao_Inadimplentes'] != 0:
        dados_atualizados.at[i,'valor_nao_pago_com_beneficios'] = atual['Carteira_Direitos_Aquisicao_Inadimplentes'] - anterior['Carteira_Direitos_Aquisicao_Inadimplentes']
        dados_atualizados.at[i,'valor_nao_pago_sem_beneficios'] = atual['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'] - anterior['Carteira_Direitos_Sem_Aquisicao_Inadimplentes']
    else:
        continue

dados_atualizados[['ID_Participante', 'Data_Competencia', 'Carteira_Direitos_Aquisicao_Inadimplentes', 'valor_nao_pago_com_beneficios', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes', 'valor_nao_pago_sem_beneficios']].head(1000)

In [ ]:
#criando o novo valor de risco
dados_atualizados = dados_atualizados.assign(vr_sem_riscos_e_beneficios = dados_atualizados['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'] + dados_atualizados['valor_nao_pago_sem_beneficios'] + dados_atualizados['Negocios_Substituicoes_Valor'] + dados_atualizados['Negocios_Recompras_Valor'] - dados_atualizados['Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao'])
dados_atualizados  = dados_atualizados.assign(vr_com_riscos_e_beneficios = dados_atualizados['Carteira_Direitos_Aquisicao_Inadimplentes'] + dados_atualizados['valor_nao_pago_com_beneficios'] + dados_atualizados['Negocios_Substituicoes_Valor'] + dados_atualizados['Negocios_Recompras_Valor'] - dados_atualizados['Ativo_Direitos_Aquisicao_Provisao_Reducao'])

dados_atualizados = dados_atualizados.assign(vr = (dados_atualizados['vr_sem_riscos_e_beneficios'] + dados_atualizados['vr_com_riscos_e_beneficios']).div(dados_atualizados['Patrimonio_Liquido']).replace(np.inf, 0))

dados_atualizados['vr'] = dados_atualizados['vr'].apply(lambda x: x if x>=0 else 0)

dados_atualizados[['vr', 'vr_sem_riscos_e_beneficios', 'vr_com_riscos_e_beneficios']].describe()

Preciso retirar outliers e dados inconsistentes para prosseguir sem prejudicar o tratamento futuro dos dados e o valor do VR

In [ ]:
#identificando outliers e possíveis inconsistências
dados_atualizados[['ID_Participante', 'vr', 'Patrimonio_Liquido', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes', 'Carteira_Direitos_Aquisicao_Inadimplentes', 'valor_nao_pago_sem_beneficios','valor_nao_pago_com_beneficios', 'Negocios_Substituicoes_Valor', 'Negocios_Recompras_Valor', 'Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao', 'Ativo_Direitos_Aquisicao_Provisao_Reducao']][dados_atualizados['ID_Participante']==dados_atualizados['vr'].max()]

In [ ]:
#retirando outliers e inconsistências
dados_atualizados = dados_atualizados.query('Patrimonio_Liquido >= 1000000')
dados_atualizados = dados_atualizados.query('ID_Participante != 117735')
dados_atualizados['vr'].describe()

In [ ]:
#normalizando dados numéricos
cols = dados_atualizados.select_dtypes(include='number').columns

#excluindo colunas de identificadores
cols=cols.delete([0,1,2])

scaler = MinMaxScaler()
scaler.fit(dados_atualizados[cols])
dados_atualizados[cols] = scaler.transform(dados_atualizados[cols])

dados_atualizados[cols].describe()

Tratados os dados nulos e os outliers, normalizadas as variáveis numéricas e agregados os dois documentos, preciso plotar gráficos para testar a hipótese do novo valor de risco.

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 104904')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'], color='red', label='Inadimplentes_Sem')
plt.plot(fundo_quebrado['Carteira_Direitos_Aquisicao_Inadimplentes'], color='blue', label='Inadimplentes_Com')
plt.plot(fundo_quebrado['Patrimonio_Liquido'], color='purple', label='PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('Inadimplência e PL x Tempo')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 104904')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['vr'], color='red', label='VR PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('VR x Tempo')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 190151')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'], color='red', label='Inadimplentes_Sem')
plt.plot(fundo_quebrado['Carteira_Direitos_Aquisicao_Inadimplentes'], color='blue', label='Inadimplentes_Com')
plt.plot(fundo_quebrado['Patrimonio_Liquido'], color='purple', label='PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('Inadimplência e PL x Tempo')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 190151')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['vr'], color='red', label='VR PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('VR x Tempo')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 133664')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['Carteira_Direitos_Sem_Aquisicao_Inadimplentes'], color='red', label='Inadimplentes_Sem')
plt.plot(fundo_quebrado['Carteira_Direitos_Aquisicao_Inadimplentes'], color='blue', label='Inadimplentes_Com')
plt.plot(fundo_quebrado['Patrimonio_Liquido'], color='purple', label='PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('Inadimplência e PL x Tempo')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
fundo_quebrado = dados_atualizados.query('ID_Participante == 133664')
plt.figure(figsize=(15, 6))
plt.plot(fundo_quebrado['vr'], color='red', label='VR PL')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.title('VR x Tempo')
plt.legend()
plt.grid(True)
plt.show()

Matrizes de correlação

In [ ]:
cols = ['vr', 'Carteira_Comercial', 'Carteira_Outros_Financeiro','Ativo_Cotas_Fundos_ICVM_555', 'Ativo_Valores_Mobiliarios', 'Ativo_Titulos_Federais', 'Ativo_Outros', 'Ativo_Outros_Ativos_Renda_Fixa', 'Ativo_Operacoes_Compromissadas', 'Ativo_Longo Prazo', 'Ativo_Disponibilidades', 'Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Esperado_Subordinada', 'Desempenho_Realizado_Subordinada', 'Valor_Cota_Senior', 'Rentabilidade_Senior', 'Desempenho_Esperado_Senior', 'Desempenho_Realizado_Senior', 'Numero_Cotistas_Senior', 'Quantidade_Cotas_Senior', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada']
data = dados_atualizados.query('ID_Participante == 92126')[cols]
plt.figure(figsize=(25,13))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
cols = ['vr', 'Carteira_Industrial', 'Carteira_Mercado_Imobiliario', 'Carteira_Comercial', 'Carteira_Comercial_Total', 'Carteira_Comercial_Varejo', 'Carteira_Servicos_Total', 'Carteira_Servicos', 'Ativo_Disponibilidades', 'Ativo_Cotas_Fundos_ICVM_555', 'Ativo_Valores_Mobiliarios', 'Ativo_Titulos_Federais','Ativo_Letras Financeiras', 'Ativo_Titulos_Federais', 'Ativo_Outros_Valores_Mobiliarios', 'Ativo_Debentures', 'Ativo_CRI', 'Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Realizado_Subordinada', 'Valor_Cota_Senior', 'Rentabilidade_Senior', 'Desempenho_Realizado_Senior', 'Numero_Cotistas_Senior', 'Quantidade_Cotas_Senior', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada']
data = dados_atualizados.query('ID_Participante == 157489')[cols]
plt.figure(figsize=(25,13))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
cols = ['vr', 'Carteira_Comercial_Total', 'Carteira_Comercial', 'Ativo_Titulos_Federais', 'Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Esperado_Subordinada', 'Desempenho_Realizado_Subordinada', 'Numero_Cotistas_Senior', 'Quantidade_Cotas_Senior', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada']
data = dados_atualizados.query('ID_Participante == 129648')[cols]
plt.figure(figsize=(15,8))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
cols = ['vr', 'Ativo_Curto_Prazo', 'Ativo_Longo Prazo', 'Ativo_Outros_Valores_Mobiliarios', 'Ativo_Valores_Mobiliarios', 'Ativo_Outros', 'Ativo_Outros_Ativos_Renda_Fixa', 'Carteira_Comercial', 'Carteira_Comercial_Total','Carteira_Outros_Financeiro', 'Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Esperado_Subordinada', 'Desempenho_Realizado_Subordinada', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada']
data = dados_atualizados.query('ID_Participante == 133664')[cols]
plt.figure(figsize=(25,13))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
cols = ['vr','Carteira_Comercial_Total', 'Carteira_Comercial', 'Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Esperado_Subordinada', 'Desempenho_Realizado_Subordinada', 'Valor_Cota_Senior', 'Rentabilidade_Senior', 'Desempenho_Esperado_Senior', 'Desempenho_Realizado_Senior', 'Numero_Cotistas_Senior', 'Quantidade_Cotas_Senior', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada','Ativo_Titulos_Federais', 'Ativo_Outros', 'Ativo_Curto_Prazo', 'Ativo_Longo Prazo', 'Ativo_Operacoes_Compromissadas', 'Ativo_Cotas_Fundos_ICVM_555', 'Ativo_Valores_Mobiliarios']
data = dados_atualizados.query('ID_Participante == 171608')[cols]
plt.figure(figsize=(25,13))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()

In [ ]:
cols = ['vr','Valor_Cota_Subordinada', 'Rentabilidade_Subordinada', 'Desempenho_Esperado_Subordinada', 'Desempenho_Realizado_Subordinada', 'Valor_Cota_Senior', 'Rentabilidade_Senior', 'Desempenho_Esperado_Senior', 'Desempenho_Realizado_Senior', 'Numero_Cotistas_Senior', 'Quantidade_Cotas_Senior', 'Numero_Cotistas_Subordinada', 'Quantidade_Cotas_Subordinada','Ativo_Titulos_Federais', 'Ativo_Valores_Mobiliarios', 'Ativo_Outros', 'Ativo_Curto_Prazo', 'Ativo_Cotas_FIDC', 'Ativo_Cotas_Fundos_ICVM_555']
data = dados_atualizados.query('ID_Participante == 173232')[cols]
plt.figure(figsize=(25,13))
sb.heatmap(data.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlação')
plt.show()